In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars  import col as c
import os 
import numpy as np
import math
from datetime import timedelta, datetime, timezone
from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from optimization_model.input_data_prepocessing import generate_first_problem_input_data
from data_display.first_stage_optimization_plots import plot_first_stage_summarized
from utility.pyomo_preprocessing import extract_optimization_results
from utility.polars_operation import linear_interpolation_for_bound, arange_float
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import generate_datetime_index, generate_clean_timeseries, optimal_segments, generate_segments, process_performance_table
from config import settings
from utility.general_function import pl_to_dict

from optimization_model.first_model_baseline_2 import generate_baseline_model

In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = '/home/ltomasini/gruobi_license/gurobi.lic'


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
.duckdb_to_schema(file_path=output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:00<00:00, 23.47it/s]


In [6]:

market_country = "CH"
market = "DA"
n_segments = 5
nb_days = 3
year = 2020
max_flow_factor: float = 1
min_flow_factor: float = 0
second_time_delta: timedelta = timedelta(minutes=60)
first_time_delta: timedelta = timedelta(days=nb_days)
min_datetime = datetime(year, 1, 1, tzinfo=timezone.utc)
max_datetime = datetime(year + 1, 1, 1, tzinfo=timezone.utc)


performance_table_per_volume, performance_table_per_state = process_performance_table(
    small_flex_input_schema=small_flex_input_schema, power_plant_name="Aegina hydro", state=[True, False, False])
        
first_datetime_index, second_datetime_index = generate_datetime_index(
        min_datetime=min_datetime, max_datetime=max_datetime, first_time_delta=first_time_delta, second_time_delta=second_time_delta)

market_price_measurement:pl.DataFrame = small_flex_input_schema.market_price_measurement\
    .filter(c("country") == market_country).filter(c("market") == market)
    
discharge_flow_measurement: pl.DataFrame = small_flex_input_schema.discharge_flow_measurement\
    .filter(c("river") == "Griessee")\
    .with_columns(
        (c("value") * timedelta(hours=1).total_seconds()).alias("discharge_volume")
    )
    
discharge_volume: pl.DataFrame = generate_clean_timeseries(
    data=discharge_flow_measurement, datetime_index=first_datetime_index,
    col_name="discharge_volume", min_datetime=min_datetime,
    max_datetime=max_datetime, time_delta=first_time_delta, agg_type="sum")

market_price: pl.DataFrame = generate_clean_timeseries(
    data=market_price_measurement, datetime_index=first_datetime_index,
    col_name="avg", min_datetime=min_datetime, 
    max_datetime=max_datetime, time_delta=first_time_delta, agg_type="mean")
    
start_basin_volume: float = performance_table_per_volume["volume"][-2]
rated_flow: float = performance_table_per_state["flow"][0][0]*3600
data: dict = {}

sets: dict = {
    "T": first_datetime_index["index"].to_list(),
    "H": performance_table_per_state["index"].to_list(),
}

constant_params: dict =  {
    "t_max": first_datetime_index.height - 1,
    "h_max": n_segments - 1,
    "max_flow": rated_flow*max_flow_factor,
    "min_flow": rated_flow*min_flow_factor,
    "start_basin_volume": start_basin_volume,
}
    
set_params: dict = {
    "max_basin_volume": pl_to_dict(performance_table_per_state.select("index", c("volume").list.get(1))),
    "min_basin_volume": pl_to_dict(performance_table_per_state.select("index", c("volume").list.get(0))),
    "alpha": pl_to_dict(performance_table_per_state[["index", "mean_alpha"]]),
    "market_price": pl_to_dict(market_price[["index", "avg"]]),
    "nb_hours": pl_to_dict(first_datetime_index[["index", "n_index"]]),
    "discharge_volume": pl_to_dict(discharge_volume[["index", "discharge_volume"]])
}
    
data.update(dict(map(lambda set: (set[0], {None: set[1]}), sets.items())))
data.update(dict(map(lambda constant_param: (constant_param[0], {None: constant_param[1]}), constant_params.items())))
data.update(set_params)
model: pyo.AbstractModel = generate_baseline_model()
solver = pyo.SolverFactory('gurobi')
model_instance = model.create_instance({None: data})
res = solver.solve(model_instance, load_solutions=True, tee=False)

plot_first_stage_summarized(model_instance=model_instance, min_flow_factor=min_flow_factor, max_flow_factor=max_flow_factor, nb_days=nb_days, year=year)


shape: (1, 2)
┌────────────────┬──────────────────┐
│ powered_volume ┆ discharge_volume │
│ ---            ┆ ---              │
│ f64            ┆ f64              │
╞════════════════╪══════════════════╡
│ 23.820974      ┆ 23.820974        │
└────────────────┴──────────────────┘
